# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import math

import matplotlib.pyplot as plt
import numpy as np
import awkward as ak
import pandas as pd
import dask
from tqdm import tqdm

import vector
import particle
import hepunits

import comet_ml
import zuko
import torch
from torch import nn, optim
import lightning as L
from lightning.pytorch import loggers as pl_loggers
from torch.utils.data import DataLoader, Subset

import multiprocessing
import uuid

from memflow.dataset.data import ParquetData
from memflow.dataset.dataset import CombinedDataset
from memflow.ttH.ttH_dataclasses import ttHHardDataset, ttHRecoDataset

from memflow.ttH.models.TransferCFM import StandardCFM as TransferCFM
from memflow.ttH.models.Transfusion import StandardCFM as Transfusion
from memflow.ttH.models.ParallelTransfusion import StandardCFM as ParallelTransfusion
from memflow.ttH.models.TransferCFM_original import StandardCFM as OriginalCFM

from memflow.ttH.distribution_plots import *
from models.utils import load_samples, save_samples
from models.callbacks import SamplingCallback, BiasCallback

from transfer_flow.transfer_flow_model import *
from transfer_flow.custom_flows import *

vector.register_awkward()

num_workers = min(16, multiprocessing.cpu_count())  # Use up to 16 CPU cores
print(f'Number of CPU workers for dataloading: {num_workers}')
os.environ["CUDA_VISIBLE_DEVICES"] = "3"  # Change "<n>" to the index of the GPU you want to use on node

print (f"Running on GPU : {torch.cuda.is_available()}")
accelerator = 'cuda' if torch.cuda.is_available() else 'cpu'
print (f"Accelerator : {accelerator}")
torch.set_float32_matmul_precision('medium')
if accelerator =='cuda':
    torch.cuda.empty_cache()
    print (torch.cuda.memory_summary(device=None, abbreviated=True))

# Data

In [ ]:
data_hard = ParquetData(
    files = [
        '/cephfs/dice/users/sa21722/datasets/MEM_data/ttH/TF_v6/hard/2018/ttH/ttH_HToInvisible_M125.parquet',
        #'all_jets_fullRun2_ttHbb_forTraining_allyears_spanetprov_part1_validation.parquet',
        #'all_jets_fullRun2_ttHTobb_forTraining_2016_PreVFP_v3.parquet',
    ],
    lazy = True,
    # N = int(1e5),
)

print (data_hard)

# Hard dataset

In [ ]:
hard_dataset = ttHHardDataset(
    data = data_hard,
    selection = [
        # 'higgs',
        # 'tops',
        'bottoms',
        # 'Ws',
        # 'Zs',
        'quarks',
        'neutrinos',
    ],
    build = False,
    fit = True,
    coordinates = 'cylindrical',
    apply_preprocessing = True,
    apply_boost = False,
    dtype = torch.float32,
)

print(hard_dataset)

# Reco dataset

In [ ]:
data_reco = ParquetData(
    files = [
        '/cephfs/dice/users/sa21722/datasets/MEM_data/ttH/TF_v6/reco/2018/ttH/ttH_HToInvisible_M125.parquet',
    ],
    lazy = True,
    #N = data_hard.N,
)

print(data_reco)

Have a look at athe minimum values for Jet and MET pT in the raw dataset. This can give an indication as to what the cutoff in the SR is and hence what to set the `'pt':lowercutshift()` to in the pre-processing

In [ ]:
reco_dataset = ttHRecoDataset(
    data = data_reco,
    selection = [
        'jets',
        'met',
    ],
    build = False,
    fit = True,
    coordinates = 'cylindrical',
    apply_preprocessing = True,
    apply_boost = False,
    dtype = torch.float32,
)
print(reco_dataset)

# Combined dataset

In [ ]:
combined_dataset = CombinedDataset(
    hard_dataset=hard_dataset,
    reco_dataset=reco_dataset,
)

train_frac = 0.8
indices = torch.arange(len(combined_dataset))
sep = int(train_frac*len(combined_dataset))
valid_indices = indices[sep:]
combined_dataset_valid = torch.utils.data.Subset(combined_dataset,valid_indices)
print (f'Dataset : validation {len(combined_dataset_valid)}')


In [ ]:
TransferCFM_model = TransferCFM.load_from_checkpoint(checkpoint_path="trained_model_checkpoints/TransferCFM_checkpoints/model_epoch_500.ckpt")
TransferCFM_model.to(accelerator)
TransferCFM_model.eval()

In [ ]:
ParallelTransfusion_model = ParallelTransfusion.load_from_checkpoint(checkpoint_path="trained_model_checkpoints/parallel_transfusion_checkpoints/model_epoch_500.ckpt")
ParallelTransfusion_model.to(accelerator)
ParallelTransfusion_model.eval()

In [ ]:
transfermer_model = TransferFlow.load_from_checkpoint(
    checkpoint_path="trained_model_checkpoints/TransferFlow_checkpoints/model_epoch_500.ckpt",
    encoder_embeddings=MultiEmbeddings(
        features_per_type=combined_dataset.hard_dataset.input_features,
        embed_dims=[32, 64],
        hidden_activation=nn.GELU,
    ),
    decoder_embeddings=MultiEmbeddings(
        features_per_type=combined_dataset.reco_dataset.input_features,
        embed_dims=[32, 64],
        hidden_activation=nn.GELU,
    ),
    transformer=Transformer(
        d_model=64,
        encoder_layers=6,
        decoder_layers=8,
        nhead=8,
        dim_feedforward=256,
        activation=nn.GELU,
        encoder_mask_attn=None,
        decoder_mask_attn=combined_dataset.reco_dataset.attention_mask,
        use_null_token=True,
        dropout=0.0,
    ),
    flow=KinematicFlow(
        d_model=64,
        flow_mode='global',
        flow_features=[
            ['pt', 'eta', 'phi', 'mass'],  # jets
            ['pt', 'phi'],  # met
        ],
        flow_classes={
            'pt': zuko.flows.NSF,
            'eta': UniformNSF,
            'phi': UniformNCSF,
            'mass': zuko.flows.NSF,
        },
        flow_common_args={
            'bins': 16,
            'transforms': 5,
            'randperm': True,
            'passes': None,
            'hidden_features': [256] * 3,
        },
        flow_specific_args={
            'eta': {'bound': 1.0},
            'phi': {'bound': math.pi},
        },
    ),
)
transfermer_model.to(accelerator)
transfermer_model.eval()

In [18]:
# Callbacks to make plots within comet
bias = BiasCallback(
    dataset = combined_dataset_valid,               # dataset on which to evaluate bias
    preprocessing = combined_dataset.reco_dataset.preprocessing, # preprocessing pipeline to draw raw variables
    N_sample = 2,                                 # number of samples to draw
    steps = 20,                                     # Number of bridging steps
    store_trajectories = False,                     # To save trajectories plots
    frequency = 50,                                 # plotting frequency (epochs)
    bins = 101,                                      # 1D/2D plot number of bins
    points = 20,                                    # Number of points for the quantile
    log_scale = True,                               # log
    batch_size = 100,                              # Batch size to evaluate the dataset (internally makes a loaded)
    #N_batch = 20,                                   # Stop after N batches (makes it faster)
    suffix = 'ttH',                                 # name for plots
    label_names = {                             # makes nicer labels
        'pt' : 'p_T',
        'eta' : '\eta',
        'phi' : '\phi',
    },
)

sampling = SamplingCallback(
    dataset = combined_dataset_valid,           # dataset to check sampling
    preprocessing = combined_dataset.reco_dataset.preprocessing, # preprocessing pipeline
    idx_to_monitor = [1,2,3,4,5,6,7,8,9,10],               # idx of events in dataset to make plots with
    N_sample = 10000,                          # number of samples to draw
    steps = 20,                                     # Number of bridging steps
    store_trajectories = False,                     # To save trajectories plots
    frequency = 50,                             # plotting frequency (epochs)
    bins = 31,                                  # 1D/2D plot number of bins
    log_scale = True,                           # log
    label_names = {                             # makes nicer labels
        'pt' : r'$p_T$ [GeV]',
        'eta' : r'$\eta$',
        'phi' : r'$\phi$ [rad]',
    },
    pt_range = 350,
)

# Transfermer per event sampling

In [ ]:
transfermer_samples_file = "Transfermer_samples2.pt"

if os.path.exists(os.path.join("saved_samples", transfermer_samples_file)):
    transfermer_samples = load_samples(transfermer_samples_file)
else:
    device = transfermer_model.device
    sampling.set_idx(sampling.idx_to_monitor)

    # Move input data to the correct device
    hard_data = [d.to(device) for d in sampling.batch['hard']['data']]
    hard_mask = [m.to(device) for m in sampling.batch['hard']['mask']]
    reco_data = [d.to(device) for d in sampling.batch['reco']['data']]
    reco_mask = [m.to(device) for m in sampling.batch['reco']['mask']]

    print(f"Hard data batch size: {hard_data[0].shape[0]}")
    print(f"Reco data batch size: {reco_data[0].shape[0]}")

    # Number of samples per event
    total_samples = sampling.N_sample  
    batch_size = 10  # Number of samples per batch to avoid memory issues
    num_batches = (total_samples + batch_size - 1) // batch_size  # Ensure full coverage

    # Storage for results
    accumulated_samples = [None, None]  

    with torch.no_grad():
        for batch_idx in tqdm(range(num_batches), desc="Sampling batches", unit="batch"):
            # Determine the number of samples to generate in this batch
            current_N = min(batch_size, total_samples - batch_idx * batch_size)

            # Generate samples
            batch_samples = transfermer_model.sample(
                hard_data,
                hard_mask,
                reco_data,
                reco_mask,
                N=current_N
            )

            # Feature selection
            jets_indices = [0, 1, 2, 3]  # ['pt', 'eta', 'phi', 'mass']
            met_indices = [0, 2]         # ['pt', 'phi']

            batch_samples[0] = batch_samples[0][..., jets_indices]  # Filter jet features
            batch_samples[1] = batch_samples[1][..., met_indices]   # Filter MET features

            # Accumulate results
            if accumulated_samples[0] is None:
                accumulated_samples[0] = batch_samples[0].cpu()
                accumulated_samples[1] = batch_samples[1].cpu()
            else:
                accumulated_samples[0] = torch.cat((accumulated_samples[0], batch_samples[0].cpu()), dim=0)
                accumulated_samples[1] = torch.cat((accumulated_samples[1], batch_samples[1].cpu()), dim=0)

    # Convert final results to the expected format
    transfermer_samples = accumulated_samples

    # Debugging prints
    num_items = len(transfermer_samples)
    print(f"Number of elements per sample: {num_items}")
    for i, sample in enumerate(transfermer_samples):
        print(f"Sample {i}: {sample.shape}")

    # Save the concatenated samples
    save_samples(transfermer_samples, transfermer_samples_file)

# 3mins 27s, 10 events, 10000 samples


In [ ]:
figures = sampling.make_sampling_plots(ParallelTransfusion_model,external_samples=transfermer_samples, cmap='RdPu', save_dir='sampling_plots/transfermer')

# Parallel Transfusion per event sampling

In [ ]:
PT_samples_file = "PT_samples2.pt"
if os.path.exists(os.path.join("saved_samples", PT_samples_file)):
    PT_samples = load_samples(PT_samples_file)
else:
    device = ParallelTransfusion_model.device
    sampling.set_idx(sampling.idx_to_monitor)
    hard_data = [d.to(device) for d in sampling.batch['hard']['data']]
    hard_mask = [m.to(device) for m in sampling.batch['hard']['mask']]
    reco_data = [d.to(device) for d in sampling.batch['reco']['data']]
    reco_mask = [m.to(device) for m in sampling.batch['reco']['mask']]

    print(f"Hard data batch size: {hard_data[0].shape[0]}")
    print(f"Reco data batch size: {reco_data[0].shape[0]}")

    with torch.no_grad():
        model = ParallelTransfusion_model.to(ParallelTransfusion_model.device)
        PT_samples = model.sample(
                            hard_data, hard_mask,
                            reco_data, reco_mask,
                            sampling.N_sample,
                            sampling.steps,
                            sampling.store_trajectories
                        )
    # Debugging prints
    num_items = len(PT_samples)
    print(f"Number of elements per sample: {num_items}")
    for i, sample in enumerate(PT_samples):
        print(f"Sample {i}: {sample.shape}")
    save_samples(PT_samples, PT_samples_file)

# 42mins 14s for 10 events, 10000 samples

In [ ]:
figures = sampling.make_sampling_plots(ParallelTransfusion_model,external_samples=PT_samples, cmap='BuGn', save_dir='sampling_plots/parallel_transfusion')

# Transfer-CFM per event sampling

In [ ]:
TransferCFM_samples_file = "TransferCFM_samples2.pt"
if os.path.exists(os.path.join("saved_samples", TransferCFM_samples_file)):
    TransferCFM_samples = load_samples(TransferCFM_samples_file)
else:
    device = ParallelTransfusion_model.device
    sampling.set_idx(sampling.idx_to_monitor)
    hard_data = [d.to(device) for d in sampling.batch['hard']['data']]
    hard_mask = [m.to(device) for m in sampling.batch['hard']['mask']]
    reco_data = [d.to(device) for d in sampling.batch['reco']['data']]
    reco_mask = [m.to(device) for m in sampling.batch['reco']['mask']]

    print(f"Hard data batch size: {hard_data[0].shape[0]}")
    print(f"Reco data batch size: {reco_data[0].shape[0]}")

    with torch.no_grad():
        model = TransferCFM_model.to(TransferCFM_model.device)
        TransferCFM_samples = model.sample(
                            hard_data, hard_mask,
                            reco_data, reco_mask,
                            sampling.N_sample,
                            sampling.steps,
                            sampling.store_trajectories
                        )
    # Debugging prints
    num_items = len(TransferCFM_samples)
    print(f"Number of elements per sample: {num_items}")
    for i, sample in enumerate(TransferCFM_samples):
        print(f"Sample {i}: {sample.shape}")
    save_samples(TransferCFM_samples, TransferCFM_samples_file)

# 11mins 25s for 10 events, 10000 samples

In [ ]:
figures = sampling.make_sampling_plots(TransferCFM_model,external_samples=TransferCFM_samples, cmap='BuPu', save_dir='sampling_plots/transferCFM')

# Bias Plots

In [19]:
device = transfermer_model.device

transfermer_bias_samples_file = "Transfermer_bias_samples.pt"

if os.path.exists(os.path.join("saved_samples", transfermer_bias_samples_file)):
    transfermer_bias_samples = load_samples(transfermer_bias_samples_file)

else:
    accumulated_samples = [[], []] # Two types of reco-level particle
    for batch_idx, batch in tqdm(enumerate(bias.loader),desc='Predict',disable=False,leave=True,total=min(bias.N_batch,len(bias.loader)),position=0):

        # Move batch data to device
        hard_data = [data.to(device) for data in batch['hard']['data']]
        hard_mask_exist = [mask.to(device) for mask in batch['hard']['mask']]
        reco_data = [data.to(device) for data in batch['reco']['data']]
        reco_mask_exist = [mask.to(device) for mask in batch['reco']['mask']]

        # Sample
        with torch.no_grad():
            transfermer_bias_samples = transfermer_model.sample(
                hard_data, hard_mask_exist,
                reco_data, reco_mask_exist,
                bias.N_sample,
            )
            # Feature selection
            jets_indices = [0, 1, 2, 3]  # ['pt', 'eta', 'phi', 'mass']
            met_indices = [0, 2]         # ['pt', 'phi']

            transfermer_bias_samples[0] = transfermer_bias_samples[0][..., jets_indices]  # Filter jet features
            transfermer_bias_samples[1] = transfermer_bias_samples[1][..., met_indices]   # Filter MET features

        # Accumulate samples across batches
        accumulated_samples[0].append(transfermer_bias_samples[0].cpu())  # Move to CPU to free GPU memory
        accumulated_samples[1].append(transfermer_bias_samples[1].cpu())

    # Concatenate accumulated samples along the batch dimension
    transfermer_bias_samples = [torch.cat(accumulated_samples[0], dim=1),  # Concatenate along batch axis
                                torch.cat(accumulated_samples[1], dim=1)]
    save_samples(transfermer_bias_samples, transfermer_bias_samples_file)

Predict:   0%|          | 0/184 [00:00<?, ?it/s]

Predict: 100%|██████████| 184/184 [00:43<00:00,  4.27it/s]

Samples saved to saved_samples/Transfermer_bias_samples.pt


In [20]:
figs = bias.make_bias_plots(ParallelTransfusion_model,show=True,external_samples=transfermer_bias_samples)

Predict: 100%|██████████| 184/184 [00:00<00:00, 333.07it/s]


IndexError: The shape of the mask [36728] at index 0 does not match the shape of the indexed tensor [6757952, 1] at index 0

## Parallel Transfusion Bias

In [ ]:
device = ParallelTransfusion_model.device

PT_bias_samples_file = "PT_bias_samples.pt"

if os.path.exists(os.path.join("saved_samples", PT_bias_samples_file)):
    PT_bias_samples = load_samples(PT_bias_samples_file)

else:
    accumulated_samples = [[], []] # Two types of reco-level particle
    for batch_idx, batch in tqdm(enumerate(bias.loader),desc='Predict',disable=False,leave=True,total=min(bias.N_batch,len(bias.loader)),position=0):

        # Move batch data to device
        hard_data = [data.to(device) for data in batch['hard']['data']]
        hard_mask_exist = [mask.to(device) for mask in batch['hard']['mask']]
        reco_data = [data.to(device) for data in batch['reco']['data']]
        reco_mask_exist = [mask.to(device) for mask in batch['reco']['mask']]

        # Sample
        with torch.no_grad():
            PT_bias_samples = ParallelTransfusion_model.sample(
                hard_data, hard_mask_exist,
                reco_data, reco_mask_exist,
                bias.N_sample,
                bias.steps,
                bias.store_trajectories
            )

        # Accumulate samples across batches
        accumulated_samples[0].append(PT_bias_samples[0].cpu())  # Move to CPU to free GPU memory
        accumulated_samples[1].append(PT_bias_samples[1].cpu())

    # Concatenate accumulated samples along the batch dimension
    PT_bias_samples = [torch.cat(accumulated_samples[0], dim=1),  # Concatenate along batch axis
                       torch.cat(accumulated_samples[1], dim=1)]

    save_samples(PT_bias_samples, PT_bias_samples_file)

Predict:  18%|█▊        | 33/184 [00:19<01:27,  1.72it/s]

In [ ]:
figs = bias.make_bias_plots(ParallelTransfusion_model,show=True,external_samples=PT_bias_samples)